Работа с API

In [4]:
!pip install requests pandas # Устанавливаем библиотеку

In [5]:
# 1. Получаем информацию обо всех книгах

import requests
import pandas as pd

# Получаем все книги
books_url = 'https://www.anapioficeandfire.com/api/books'
response = requests.get(books_url)
books = response.json()

# Сохраняем в DataFrame
books_df = pd.DataFrame(books)
print(books_df.head())

# 2. Получаем информацию обо всех домах Вестероса
# API возвращает данные по 10 домов на страницу. Проходимся по всем страницам.

houses = []
page = 1

while True:
    houses_url = f'https://www.anapioficeandfire.com/api/houses?page={page}&pageSize=50'
    response = requests.get(houses_url)
    data = response.json()
    if not data:
        break
    houses.extend(data)
    page += 1

houses_df = pd.DataFrame(houses)
print(houses_df.head())

# 3. Получаем дома, у которых есть девиз (hasWords=true)

houses_with_words = []
page = 1

while True:
    houses_url = f'https://www.anapioficeandfire.com/api/houses?page={page}&pageSize=50&hasWords=true'
    response = requests.get(houses_url)
    data = response.json()
    if not data:
        break
    houses_with_words.extend(data)
    page += 1

houses_with_words_df = pd.DataFrame(houses_with_words)
print(houses_with_words_df[['name', 'words']].head())

                                             url               name  \
0  https://www.anapioficeandfire.com/api/books/1  A Game of Thrones   
1  https://www.anapioficeandfire.com/api/books/2   A Clash of Kings   
2  https://www.anapioficeandfire.com/api/books/3  A Storm of Swords   
3  https://www.anapioficeandfire.com/api/books/4   The Hedge Knight   
4  https://www.anapioficeandfire.com/api/books/5  A Feast for Crows   

             isbn                authors  numberOfPages  \
0  978-0553103540  [George R. R. Martin]            694   
1  978-0553108033  [George R. R. Martin]            768   
2  978-0553106633  [George R. R. Martin]            992   
3  978-0976401100  [George R. R. Martin]            164   
4  978-0553801507  [George R. R. Martin]            784   

                   publisher        country     mediaType  \
0               Bantam Books  United States     Hardcover   
1               Bantam Books  United States      Hardback   
2               Bantam Books  Unite

Работа с БД

In [6]:
!pip install psycopg2-binary # Устанавливаем библиотеку

import psycopg2
import pandas as pd

In [7]:
# Подключение к базе данных RNAcentral
conn = psycopg2.connect(
    host="hh-pgsql-public.ebi.ac.uk",
    database="pfmegrnargs",
    user="reader",
    password="NWDMCE5xdipIjRrp",
    port=5432
)

## 1. Получение 10 строк из таблицы rnc_database
with conn.cursor() as cursor:
    # Выполняем SQL-запрос
    cursor.execute("SELECT * FROM rnc_database LIMIT 10")

    # Получаем результаты
    results = cursor.fetchall()

    # Создаем DataFrame с названиями столбцов
    columns = [desc[0] for desc in cursor.description]
    df_all_columns = pd.DataFrame(results, columns=columns)

print("Все столбцы:")
print(df_all_columns.head())

## 2. Получение конкретных столбцов
with conn.cursor() as cursor:
    # Выполняем запрос с выбором конкретных полей
    cursor.execute("""
        SELECT display_name, num_sequences, num_organisms, url
        FROM rnc_database
        LIMIT 10
    """)

    # Получаем результаты
    results = cursor.fetchall()

    # Создаем DataFrame
    df_specific = pd.DataFrame(
        results,
        columns=['display_name', 'num_sequences', 'num_organisms', 'url']
    )

# Закрываем соединение
conn.close()

print("\nВыбранные столбцы:")
print(df_specific)

Все столбцы:
   id                  timestamp userstamp      descr  current_release  \
0   1 2017-05-01 00:00:00.000000    RNACEN        ENA              884   
1   5 2017-05-17 00:00:00.000000    RNACEN       VEGA               98   
2  26 2017-05-01 00:00:00.000000    RNACEN    GENCODE              450   
3  55 2023-10-10 15:02:45.191606    RNACEN     MGNIFY              839   
4  41 2017-05-01 00:00:00.000000    RNACEN  GENECARDS              867   

  full_descr alive for_release display_name project_id  avg_length  \
0        ENA     Y                      ENA                  412.0   
1       VEGA     N                     VEGA  PRJEB4568         NaN   
2    GENCODE     N                  GENCODE                  889.0   
3     MGnify     Y        None       MGnify       None       151.0   
4  MalaCards     Y                GeneCards                 1298.0   

   min_length  max_length  num_sequences  num_organisms  \
0        10.0    900074.0       12086180         814855   
1  